In [1]:
import pickle
import pandas as pd
import os
import csv
import shutil
import collections
import csv
from IPython.core.display import HTML
import gensim
import pickle
import nltk
import os
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
#nltk.download()
import string 
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import collections 

pd.set_option('display.max_rows', None)

with open('id2gram.pkl', 'rb') as f:
    listofgrams = pickle.load(f)
print(len(listofgrams))

#case metadata
casedata = pd.read_pickle('casedata.pkl')

#taking only year 2013
yearr="2013"
df=casedata[(casedata["year"] == yearr)]
print(len(df))

#list of n grams present in firm related cases of 2013
listofgrams_law={}
 
for i in range(1,13):
    pathname = os.path.join("PickleFiles/"+yearr, str(i))
    files=os.listdir(pathname)
    for file in files:
        file1=file.split("_")[0]
        caseid=file1
        
        path=pathname+"/"+file
        with open(path, 'rb') as m:
            mapping = pickle.load(m)
       
        
        for sent in mapping:
            for token in sent:
                if token in listofgrams:
                    listofgrams_law[listofgrams[token]]=token
                else:
                    print("error")
print(len(listofgrams_law))
#listofgrams_law

96977
481
42862


In [2]:

#check df tf of firm related bigrams
fewgrams={}
with open('df-tf.pkl', 'rb') as f:
    dftf = pickle.load(f)
print(len(dftf))

df1=dftf['phrase'].tolist()
i=0
for bigram in df1:
    if bigram in  listofgrams_law:
        tt=dftf.loc[dftf['phrase'] == bigram, 'termfreqs'].item()
        
        fewgrams[bigram]=tt
    else:
        i=i+1
        
print(i)     
print(len(fewgrams))
#fewgrams

100495
57633
42862


In [3]:
#taking bigrams with specific dftf
finalgrams={}
for f in fewgrams:
    if fewgrams[f]<9000 and fewgrams[f]>800:
        finalgrams[f]=listofgrams_law[f]


print(len(finalgrams))
#finalgrams

12205


In [4]:
#final dataframe
columns1=['CaseId','Date','Month']


for f in finalgrams:
    columns1.append(f)


ngramdata= pd.DataFrame(columns=columns1)



In [5]:
#appending rows to dataframe for each case of year 2013 

i=0
for index, row in df.iterrows():
    i=i+1
    
    caseid=index
    date=row["date"]
    month=int(row["month"])
    
    ngramdata = ngramdata.append({'CaseId':caseid,'Date':date,'Month':month},ignore_index=True)
    
print(len(ngramdata))   

ngramdata = ngramdata.set_index('CaseId')


481


In [6]:

ngramdata.info()


<class 'pandas.core.frame.DataFrame'>
Index: 481 entries, X10OE4DGG000N to XVITNB80000N
Columns: 12207 entries, Date to enrichment_claim
dtypes: object(12207)
memory usage: 44.8+ MB


In [7]:



#reading pickle files and updating the ngrams that our present 
for i in range(1,13):
    pathname = os.path.join("PickleFiles/"+yearr, str(i))
    files=os.listdir(pathname)
    for file in files:
        file1=file.split("_")[0]
        caseid=file1
        
        path=pathname+"/"+file
        with open(path, 'rb') as m:
            mapping = pickle.load(m)
       
        
        for sent in mapping:
            for token in sent:
                if token in finalgrams.values():
                    
                    ngramdata.loc[caseid,listofgrams[token]]=1
                
                    
        





In [8]:

ngramdata=ngramdata.fillna(0)


In [9]:
ngramdata.to_pickle("ngramdata.pkl")




In [10]:
with open('ngramdata.pkl', 'rb') as f:
    ngramdata = pickle.load(f)




In [11]:
train_data=ngramdata[ngramdata["Month"]<=7]
test_data=ngramdata[ngramdata["Month"]>7]

In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 337 entries, X10OE4DGG000N to X1QL5FTPG000N
Columns: 12207 entries, Date to enrichment_claim
dtypes: int64(12206), object(1)
memory usage: 31.4+ MB


In [13]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 144 entries, X1B7QSOG0000N to XVITNB80000N
Columns: 12207 entries, Date to enrichment_claim
dtypes: int64(12206), object(1)
memory usage: 13.4+ MB


In [14]:
train_data.to_pickle("ngramdata_train.pkl")
test_data.to_pickle("ngramdata_test.pkl")

In [15]:
pd.set_option('display.max_columns', 500)
train_data.head()

,Date,Month,affidavit_state,different_view,case_differs,car_wash,aggravated_murder,review_committee,correct_result,additional_defendant,regular_basis,negligent_act,montgomery_ward,former_position,public_purpose,supplemental_security,strategic_decision,political_campaign,portland_cement,section_apply,underlying_purpose,inadmissible_hearsay,actual_authority,citizen_suit,judge_john,insuring_agreement,take_custody,secretary_argue,must_balance,fourteen_day,state_street,must_provide,wrongful_conduct,amoco_oil,first_nat,fundamental_rule,may_maintain,party_intention,would_face,american_airline,counsel_appoint,rulemaking_authority,public_reputation,various_type,trustee_could,drug_relate,allow_state,see_washington,management_service,may_indicate,common_control,beam_circuit,would_rely,respective_interest,effect_would,recent_case,legal_analysis,additional_ground,account_would,risk_pose,coal_company,first_judgment,indian_reservation,say_petition,independent_claim,proximate_causation,chinese_government,new_business,would_reveal,policy_embody,standing_issue,may_petition,prior_statement,telephone_company,thus_appellant,mere_existence,whose_right,private_individual,financial_responsibility,exclusive_mean,separate_contract,legal_proceeding,assistance_claim,period_allow,taxpayer_file,back_seat,foregoing_analysis,american_court,supplemental_jurisdiction,judicial_opinion,statutory_mandate,jurisdictional_basis,twenty_percent,cognizable_claim,cir_friendly,law_analysis,arm_length,would_exclude,first_state,reasonable_measure,filing_suit,commissioner_may,allow_interest,board_lack,state_license,rule_permit,activity_involve,introducing_evidence,limitation_bar,disabled_person,propose_intervenor,act_include,judicial_record,improper_motive,solicitor_general,teamster_local,court_charge,acquire_title,saving_bank,actual_value,state_would,view_would,prior_case,voluntary_dismissal,additional_share,conduct_cause,textron_inc,drug_use,exceptional_case,ark_code,information_could,defendant_choose,assess_damage,second_element,contribution_claim,first_congress,justice_jackson,could_qualify,secretary_could,company_appeal,would_determine,health_review,concerted_action,federal_energy,dismissing_plaintiff,would_force,erisa_case,unlawful_entry,sess_reprint,jefferson_county,seek_reversal,smith_contend,time_record,particular_matter,court_accept,state_fed,limited_number,contract_require,union_claim,remedial_statute,current_version,defendant_bear,probable_consequence,fee_petition,contempt_hearing,personal_stake,proper_amount,judicial_power,allege_breach,government_show,may_increase,medical_expense,michigan_statute,insufficient_proof,plaintiff_receive,successful_completion,federal_crime,government_expert,give_day,particular_language,amply_support,would_place,hotel_room,actual_cash,gas_elec,puerto_rican,would_impair,little_discussion,union_carbide,evidence_use,creditor_committee,trial_indicate,case_cannot,full_compliance,federal_prison,work_performance,particular_statute,prospective_purchaser,former_attorney,exclusive_possession,concurring_opinion,doubt_instruction,circumstance_test,following_exchange,plea_court,remedy_afford,cable_company,approximately_mile,would_deprive,principal_issue,willful_failure,waste_disposal,time_prescribe,federal_cause,ed_state,decide_today,act_without,precedential_value,discrimination_may,primary_argument,second_set,sufficient_consideration,baltimore_city,true_fact,must_relate,dispute_resolution,court_take,pre_deprivation,illegal_arrest,congress_authorize,certified_question,age_base,act_make,defendant_enter,prior_approval,proper_resolution,union_local,common_use,deduction_claim,statutory_exemption,union_united,fundamental_fairness,would_recommend,must_meet,file_october,legal_theory,race_color,court_defer,specific_term,disability_determination,...,enter_upon,improper_statement,act_rico,ground_assert,court_united,return_verdict,work_experience,novo_determination,temporary_injunction,file_may,protection_act,case_require,appellant_appeal,arkansas_statute,

In [16]:
j=0
for i in range(0,337):
    
    t=train_data.ix[:,train_data.iloc[i] == 1].columns.values
    if len(t)==0:
        j=j+1
print(j)

/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.


0
